In [2]:
import matplotlib.pyplot as plt
# torch is just for the feature extractor and the dataset (NOT FOR IMPLEMENTING NEURAL NETWORKS!)
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.models import resnet34
import torch.nn as nn
# sklearn is just for evaluation (NOT FOR IMPLEMENTING NEURAL NETWORKS!)
from sklearn.metrics import confusion_matrix, f1_score


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
feature_extractor = resnet34(pretrained=True)
input_dim = feature_extractor.fc.in_features
for param in feature_extractor.parameters():
    param.requires_grad = False

feature_extractor.fc = nn.Identity()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 109MB/s]


In [4]:
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)


100%|██████████| 170498071/170498071 [00:02<00:00, 69512430.87it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [5]:
from torch.utils.data import DataLoader


In [6]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)


In [22]:
# importing numpy library
import numpy as np

# defining a class named RBFNet
class RBFNet:
    
    # constructor method to initialize the attributes of the object, namely num_inputs, num_hidden, num_outputs and gamma.
    def __init__(self, num_inputs, num_hidden, num_outputs, gamma=1.0):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.gamma = gamma

        # initializing weights for hidden layer with random values
        self.hidden_weights = np.random.rand(self.num_hidden, self.num_inputs)

        # initializing biases for hidden layer with random values
        self.hidden_biases = np.random.rand(self.num_hidden)

        # initializing weights for output layer with random values
        self.output_weights = np.random.rand(self.num_outputs, self.num_hidden)

        # initializing biases for output layer with random values
        self.output_biases = np.random.rand(self.num_outputs)

    # method to calculate activation function for Radial Basis Function (RBF)
    def rbf(self, x, c, s):
        return np.exp(-self.gamma * np.linalg.norm(x-c)**2 / s**2)

    # method to calculate the activation of hidden layer
    def hidden_layer_activation(self, X):
        Z = np.zeros((self.num_hidden, X.shape[0]))
        for i in range(self.num_hidden):
            for j in range(X.shape[0]):
                Z[i, j] = self.rbf(X[j], self.hidden_weights[i], 1)
        return Z.T

    # method to perform forward propagation
    def forward(self, X):
        # calculate activation of hidden layer
        hidden_layer_activation = self.hidden_layer_activation(X)

        # calculate output of hidden layer
        hidden_output = np.dot(hidden_layer_activation,
                               self.output_weights.T) + self.output_biases

        # apply softmax function to output
        output = self.softmax(hidden_output)
        return output

    # method to calculate softmax function
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / exp_x.sum(axis=1, keepdims=True)

    # method to convert the output labels into one-hot encoded vectors
    def one_hot(self, y):
        onehot = np.zeros((y.size, self.num_outputs))
        onehot[np.arange(y.size), y] = 1
        return onehot

    # method to make prediction using the trained model
    def predict(self, X):
        # calculate activation of hidden layer
        hidden_layer_activation = self.hidden_layer_activation(X)

        # calculate output of hidden layer
        hidden_output = np.dot(hidden_layer_activation,
                               self.output_weights.T) + self.output_biases

        # apply softmax function to output
        output = self.softmax(hidden_output)
        
        # return the index of the maximum value in each row of the 'output' array as a predicted class label.
        return np.argmax(output, axis=1)

    # method to train the RBF network
    def train(self, X, y, lr=0.01, epochs=100):
        for epoch in range(epochs):
            
            # calculate activation of hidden layer
            hidden_layer_activation = self.hidden_layer_activation(X)

            # calculate output of hidden layer
            hidden_output = np.dot(
                hidden_layer_activation, self.output_weights.T) + self.output_biases

            # apply softmax function to output
            output = self.softmax(hidden_output)

            # calculate error between predicted and actual output
            error = self.one_hot(y) - output
            
            # calculate gradient of output weights and biases
            output_weights_gradient = - \
                np.dot(error.T, hidden_layer_activation) / X.shape[0]
            output_biases_gradient = -np.mean(error, axis=0)

            # backpropagate error to hidden layer
            hidden_error = np.dot(error, self.output_weights) * \
                (hidden_layer_activation * (1 - hidden_layer_activation))

            # calculate gradient of hidden weights and biases
            hidden_weights_gradient = -np.dot(hidden_error.T, X) / X.shape[0]
            hidden_biases_gradient = -np.mean(hidden_error, axis=0)

            # update weights and biases
            self.output_weights -= lr * output_weights_gradient
            self.output_biases -= lr * output_biases_gradient
            self.hidden_weights -= lr * hidden_weights_gradient
            self.hidden_biases -= lr * hidden_biases_gradient

            # print loss for each epoch
            loss = np.mean(np.square(error))
            if epoch % 10 == 0:
                print("Epoch %d: loss = %.6f" % (epoch, loss))


In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=1000,
                    hidden_layer_sizes=[20]).fit(result, Y)
clf.score(result, Y)


0.999375

In [24]:
rbf = RBFNet(512, 20, 10)
rbf.train(result, Y)


Epoch 0: loss = 0.090674
Epoch 10: loss = 0.090660
Epoch 20: loss = 0.090646
Epoch 30: loss = 0.090632
Epoch 40: loss = 0.090619
Epoch 50: loss = 0.090606
Epoch 60: loss = 0.090593
Epoch 70: loss = 0.090581
Epoch 80: loss = 0.090569
Epoch 90: loss = 0.090557


In [30]:
rbf.predict(result[0])[0]


3

In [31]:
counter = 0
for i, t in enumerate(result):
    if rbf.predict(t)[0] == Y[i]:
        counter += 1
print("RBF accuracy:")
print(counter / len(result))


RBF accuracy:
0.096875
